In [1]:
import pandas as pd

In [2]:
filepath = "../data/train.csv"

In [3]:
train_df = pd.read_csv(filepath, index_col = 0, nrows= None, engine = "python")

In [4]:
train_df.shape

(6331, 2)

In [5]:
train_df.head()

,review,label
ID,,
1,Jo bhi ap se tou behtar hoon,Negative
2,ya Allah meri sister Affia ki madad farma,Positive
3,Yeh khud chahta a is umar main shadi krna. ha...,Negative
4,Tc ? Apky mun xe exe alfax achy nae lgty 馃槖馃拑,Negative
5,Good,Positive


In [6]:
train_df.dtypes

review    object
label     object
dtype: object

In [7]:
train_df["num_lbl"] = train_df['label'].map({"Negative": 0, "Positive": 1})

In [8]:
train_df.head()

,review,label,num_lbl
ID,,,
1,Jo bhi ap se tou behtar hoon,Negative,0.0
2,ya Allah meri sister Affia ki madad farma,Positive,1.0
3,Yeh khud chahta a is umar main shadi krna. ha...,Negative,0.0
4,Tc ? Apky mun xe exe alfax achy nae lgty 馃槖馃拑,Negative,0.0
5,Good,Positive,1.0


In [9]:
train_df['review'].head(20)

ID
1                          Jo bhi ap se tou behtar hoon
2             ya Allah meri sister Affia ki madad farma
3     Yeh khud chahta a is umar main shadi krna.  ha...
4         Tc ? Apky mun xe exe alfax achy nae lgty 馃槖馃拑
5                                                  Good
6     American president John f Kennedy aur in ke bh...
7      Commission aur kickback ka dor Dora raha, quo...
8     Allah pak nazer e bd sy bechye or humesha boha...
9     Amoman log samajhte hain ke jhok siyal hi Abid...
10    Akki KhanYani k tum ....... v good Wesy tum sh...
11    Jail Road Pr Firing Se 1 Shaks Janbahaq Rpt M ...
12    hud hagai stupid actor hy.....acting ati he na...
13                              Haha thank you so much 
14    Pakistani cricket ki tareekh ka behtreen batsm...
15    Nawaz Sharif ko Pakro Harimi Police walo Tum l...
16                 Kash MERI MAAN ZINDA HOTI...........
17    5 Billion Dollor se zaid akhrajat ka takhmina ...
18    Kisi larki kay sat selfie lana tum hay 

In [10]:
import re

In [11]:
strlist = []
filter = re.compile("[^0-9a-z,.?:]")
for i, s in enumerate(train_df['review']):
    s_new = filter.sub(" ", s.lower());
    if(i < 500):
        print("---------**----------")
        print(s)
        print(s_new)
    strlist.append(s_new)

---------**----------
Jo bhi ap se tou behtar hoon
jo bhi ap se tou behtar hoon
---------**----------
ya Allah meri sister Affia ki madad farma
ya allah meri sister affia ki madad farma
---------**----------
Yeh khud chahta a is umar main shadi krna.  had ogi
yeh khud chahta a is umar main shadi krna.  had ogi
---------**----------
Tc ? Apky mun xe exe alfax achy nae lgty 馃槖馃拑
tc ? apky mun xe exe alfax achy nae lgty     
---------**----------
Good
good
---------**----------
American president John f Kennedy aur in ke bhai Robert Kennedy se bhi Marilyn Monroe ke affairs ka charcha raha
american president john f kennedy aur in ke bhai robert kennedy se bhi marilyn monroe ke affairs ka charcha raha
---------**----------
 Commission aur kickback ka dor Dora raha, quomi assase koriyon ke mol farokhat keye gaye
 commission aur kickback ka dor dora raha, quomi assase koriyon ke mol farokhat keye gaye
---------**----------
Allah pak nazer e bd sy bechye or humesha bohat tarkian dy Ameeen
alla

yahan bohat mutmaiyn hon
---------**----------
Peshawar walon ka to jeena haram kar dya hai without planning metro banany jarhy ho . Un k 7 hamara kya kam kpk k liye ap ny kya keya . Apna ghar smbhala nhi jata to dosro ko point out na kry wo apny amal ky jawab deh hain ap apny amal k. Allah sb ko hidayat dy. Ameen
peshawar walon ka to jeena haram kar dya hai without planning metro banany jarhy ho . un k 7 hamara kya kam kpk k liye ap ny kya keya . apna ghar smbhala nhi jata to dosro ko point out na kry wo apny amal ky jawab deh hain ap apny amal k. allah sb ko hidayat dy. ameen
---------**----------
Allah hamain tasbih khany me ana nasib ho Ameen
allah hamain tasbih khany me ana nasib ho ameen
---------**----------
 Vespa scoter se CID ke head quarter mein Prado par aane wale Chaudhry Aslam ne zindagi ke bohat se nasheeb o faraz dekhe
 vespa scoter se cid ke head quarter mein prado par aane wale chaudhry aslam ne zindagi ke bohat se nasheeb o faraz dekhe
---------**----------
Shah Fais

In [12]:
out_df = pd.DataFrame({"comment": strlist, "label": train_df["num_lbl"]})

In [13]:
out_df.head()

,comment,label
ID,,
1,jo bhi ap se tou behtar hoon,0.0
2,ya allah meri sister affia ki madad farma,1.0
3,yeh khud chahta a is umar main shadi krna. ha...,0.0
4,tc ? apky mun xe exe alfax achy nae lgty,0.0
5,good,1.0


In [14]:
out_df.to_csv("../data/train_input.csv")

In [15]:
import sys
sys.path.append("../")

In [16]:
# %load ../fopen.py
from scipy.sparse import dok_matrix, csr_matrix
import numpy as np
import pandas as pd
from time_wrapper import func_timer
import os
import random
from itertools import cycle
from config import log
import re
# from timeit import Timer



@func_timer
def fopen(filepath):
    '''
    read file aclImdb/train/labeledBow.feat to a scipy csr object
    '''
    count = 0
    x_spa = dok_matrix((25000, 89527),dtype= int)
    labels = np.ndarray(shape=(25000,),dtype= int)
    print('fopen start:')
    with open(filepath,'r') as f:
        for i, line in enumerate(f):
            # print(line)
            label, features = line.split(sep= ' ', maxsplit= 1)
            labels[i] = 0 if int(label) < 5 else 1
            for feature in features.split(sep = ' '):
                ind, value = feature.split( sep = ':')
                # print(ind, ':', value)
                x_spa[i, int(ind)] = int(value)
            # print(label, dok[i,:])
            # count += 1
            # if count == 3:
            #     break
        # x_spa = x_spa.tocsr()
        print('fopen succeed! ')
        return x_spa , labels

class Sentences():
    def __init__(self, filename, loop):
        """

        :param dirnames:
        :param loop: bool 类型, 决定是否循环利用迭代器
        """
        self.filename = filename
        self.loop = loop
        self.sentences = pd.read_csv(filename, usecols= ['comment', ], squeeze= True)

    # 这不是一个迭代器，但是一个可迭代对象。
    # 通过 self.loop 标志 确定是否循环迭代
    # 调用 self.__iter__ 得到一个生成器, 提供了一种复用生成器的方法.!!!
    def __iter__(self):
        if self.loop:
            self.sentences = cycle(self.sentences)
        for line in self.sentences:
            wordlist = re.findall("\w+'\w+|\w+|[,.:?]", line)
            yield wordlist

def corpora(filename, loop_or_not):
    """
    这是一个生成器函数, 返回一个生成器, 实现了 迭代器 协议.
    :param dirpaths:
    :param loop_or_not:
    :return:
    """
    sentences = Sentences(filename= filename, loop = loop_or_not)
    for sentence in sentences:
        yield sentence


if __name__ == '__main__':
    # x, y = fopen()
    # xi, yi = Iter_open('aclImdb/test/labeledBow.feat').__next__()
    # print(fopen.__doc__)
    # print(Iter_open.__doc__)
    # # print(x.shape, y.shape)
    # print(xi, yi)

    # test_comment = '''I haven't read a biography of Lincoln, so maybe this was an accurate portrayal......<br /><br />And maybe it's because I'm used to the equally alienating and unrealistic worshiping portrayals that unnaturally deify Lincoln as brilliant, honorable, and the savior of our country......<br /><br />But why would they make a movie representing Lincoln as a buffoon? While Henry Fonda made an excellent Lincoln, his portrayal of him as an "aw shucks, I'm just a simple guy" seemed a little insulting.<br /><br />[Granted, that was Bushie Jr.'s whole campaign, to make us think he was "just a regular guy" so we wouldn't care that he's a rich & privileged moron -- but that's a whole other story.]<br /><br />Not only did the film show Lincoln as sort of a simple (almost simple-minded) kind of guy , the film states that Lincoln just sort of got into law by accident, and that he wasn't even that interested in the law - only with the falsely simplistic idea of the law being about rights & wrongs. In the film he's not a very good defense attorney (he lounges around with his feet on the table and makes fun of the witnesses), and the outcome is mostly determined by chance/luck.<br /><br />Furthermore, partly because this was financed by Republicans (in reaction to some play sponsored by Democrats that had come out) and partly because it was just the sentiment of the times, the film is unfortunately religious, racist and conservative.<br /><br />Don't waste your time on this film.'''
    # test_comment = re.sub('<br\s*/>', '', test_comment)
    # log.debug('comment after substitute: %s.' %test_comment)
    # wordlist = [word for word in re.findall("\w+'\w+|\w+|[,.:?]", test_comment)]
    # log.debug('comment before split: %s' %test_comment)
    # log.debug('comment after split: %s' % wordlist)
    # log.debug('comment after split: %s' % ' '.join(wordlist))

    sentences = Sentences('../data/train_input.csv', loop= True)
    for count, sentence in enumerate(sentences):
        print(len(sentence))
        print(' '.join(sentence))
        if count == 20:
            break


7
jo bhi ap se tou behtar hoon
8
ya allah meri sister affia ki madad farma
12
yeh khud chahta a is umar main shadi krna . had ogi
10
tc ? apky mun xe exe alfax achy nae lgty
1
good
20
american president john f kennedy aur in ke bhai robert kennedy se bhi marilyn monroe ke affairs ka charcha raha
16
commission aur kickback ka dor dora raha , quomi assase koriyon ke mol farokhat keye gaye
13
allah pak nazer e bd sy bechye or humesha bohat tarkian dy ameeen
35
amoman log samajhte hain ke jhok siyal hi abid ali ki pheli serial thi halan ke sach ye hai ke unhon ne sab se phele saleem chishtti le likhe howe kheel zanjeer mein adakari ki
22
akki khanyani k tum . . . . . . . v good wesy tum shkll sy bhi khesiani billi kgti ho
12
jail road pr firing se 1 shaks janbahaq rpt m ali jokhio
47
hud hagai stupid actor hy . . . . . acting ati he nahi isay . . . . . . . maham . . . kiya dakh liya app nay is mein . . sayra . . . . shaadi kr loo yar . . .
5
haha thank you so much
34
pakistani cricket ki t

In [17]:
    from gensim.corpora import Dictionary
    filename = '../data/train_input.csv'
    corp = corpora(filename = filename, loop_or_not= False)
    dict = Dictionary(documents=corp)
    #dict.filter_extremes(no_below=5, no_above=0.5)

C:\Users\so_go\Anaconda3\envs\Python3_5\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2019-06-14 15:03:26,112 <module>            :   INFO: 'pattern' package not found; tag filters are not available for English
2019-06-14 15:03:26,166 add_documents       :   INFO: adding document #0 to Dictionary(0 unique tokens: [])
2019-06-14 15:03:26,427 add_documents       :   INFO: built Dictionary(17724 unique tokens: ['74', 'bhai', 'nind', 'sukoon', 'kashair']...) from 6331 documents (total 104689 corpus positions)


In [18]:
help(dict)

Help on Dictionary in module gensim.corpora.dictionary object:

class Dictionary(gensim.utils.SaveLoad, collections.abc.Mapping)
 |  Dictionary encapsulates the mapping between normalized words and their integer ids.
 |  
 |  Attributes
 |  ---------
 |  token2id : dict of (str, int)
 |      token -> tokenId.
 |  id2token : dict of (int, str)
 |      Reverse mapping for token2id, initialized in lazy manner to save memory.
 |  dfs : dict of (int, int)
 |      Document frequencies: token_id -> in how many documents contain this token.
 |  num_docs : int
 |      Number of documents processed.
 |  num_pos : int
 |      Total number of corpus positions (number of processed words).
 |  num_nnz : int
 |      Total number of non-zeroes in the BOW matrix.
 |  
 |  Method resolution order:
 |      Dictionary
 |      gensim.utils.SaveLoad
 |      collections.abc.Mapping
 |      collections.abc.Sized
 |      collections.abc.Iterable
 |      collections.abc.Container
 |      builtins.object
 |  
 |

In [19]:
token2id = dict.token2id

In [20]:
token2id["good"]

37

In [21]:
def word_freq(string):
    return dict.dfs[dict.token2id[string]]

In [22]:
word_freq("good")

80

In [25]:
test_df = pd.read_csv("../data/20190610_test.csv", engine = "python")

In [26]:
test_df.head()

,ID,review
0,1,Sarcasm acha karti ho xD
1,2,Ali tareen ki kon c study yar...... ye bhai ph...
2,3,Sharab Juwa Butt Aur Faal K Teer Shaitani Kam ...
3,4,salam hakeem sahab mne pore ramadan boht faida...
4,5,Bata ku nae dyty ake dusray ko apny dil ki baat


In [27]:
strlist = []
for i, s in enumerate(test_df['review']):
    s_new = filter.sub(" ", s.lower());
    if(i < 500):
        print("---------**----------")
        print(s)
        print(s_new)
    strlist.append(s_new)

---------**----------
Sarcasm acha karti ho xD
sarcasm acha karti ho xd
---------**----------
Ali tareen ki kon c study yar...... ye bhai phly election lrr chky hain.... ye ho skta hy..... jan bohj kr pechy hwa ho....ye btany k lye....hmy lalch ni....ap kia khty ho???
ali tareen ki kon c study yar...... ye bhai phly election lrr chky hain.... ye ho skta hy..... jan bohj kr pechy hwa ho....ye btany k lye....hmy lalch ni....ap kia khty ho???
---------**----------
Sharab Juwa Butt Aur Faal K Teer Shaitani Kam Hain In Se Bacho Takay Kamyabi Or Nijat Pao 
sharab juwa butt aur faal k teer shaitani kam hain in se bacho takay kamyabi or nijat pao 
---------**----------
salam hakeem sahab mne pore ramadan boht faida mila meri info me boht izafa hua vazaeef ka pta chala tasbeehat ka pta chala jo k me prhti hon online mureed bhi huee allah ap ko is mehnat ka ajar ata frmaeyn dua ki drkhuwast hai
salam hakeem sahab mne pore ramadan boht faida mila meri info me boht izafa hua vazaeef ka pta chala t

Surjani Town Abdullah Mor K Qareeb Se 1 Shaks Ki Lash Baramad 
surjani town abdullah mor k qareeb se 1 shaks ki lash baramad 
---------**----------
馃槀馃槀. Jo bhi pas sy guzry usy judge kro ..best skill
    . jo bhi pas sy guzry usy judge kro ..best skill
---------**----------
Eyex kaisi batain kar rhy ho...pagla gai ho kya...
eyex kaisi batain kar rhy ho...pagla gai ho kya...
---------**----------
Aysi Jaga Ko Kehty Hein Jaha Toop Rakhkar Chalaya Jata Hn 
aysi jaga ko kehty hein jaha toop rakhkar chalaya jata hn 
---------**----------
 January 1954 mein reha keye gaye to Soba Sarhad mein dakhla aur naqlo harkat ki azadi na thi
 january 1954 mein reha keye gaye to soba sarhad mein dakhla aur naqlo harkat ki azadi na thi
---------**----------
 Ap ke walid aik ilm pasand aadmi the
 ap ke walid aik ilm pasand aadmi the
---------**----------
Behan faraz manan ka lehmga pehne wle saaade se to log hn...LNG ka chota sa to ghapla hai ita to acha hai
behan faraz manan ka lehmga pehne wle saaade s

In [32]:
test_input = pd.DataFrame({"ID": test_df["ID"], "comment": strlist})

In [33]:
test_input.shape

(2712, 2)

In [34]:
test_input.to_csv("../data/test_input_0610.csv")

In [35]:
test_input["ID"].max()

2712